In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('../Data/train.csv')
df_train

,time,signal,open_channels
0,0.0001,-2.7600,0
1,0.0002,-2.8557,0
2,0.0003,-2.4074,0
3,0.0004,-3.1404,0
4,0.0005,-3.1525,0
...,...,...,...
4999995,499.9996,2.9194,7
4999996,499.9997,2.6980,7
4999997,499.9998,4.5164,8
4999998,499.9999,5.6397,9


In [3]:
signal = np.array(df_train.signal)
open_channels = np.array(df_train.open_channels)
#signals = np.reshape(signal,(1))
x = np.reshape(signal,(1000,5000,1))
y = np.reshape(open_channels,(1000,5000,1))

In [4]:
model = Sequential()
model.add(SimpleRNN(units=20, return_sequences=True, input_shape=(5000,1,)))
model.add(Dense(20,activation='relu'))
model.add(Dense(units=1,activation='relu'))
model.compile(loss='mean_squared_error',optimizer='sgd',metrics=['accuracy'])

In [5]:
model.fit(np.array(x), y, epochs=10, validation_split=.3,verbose=1)

Train on 700 samples, validate on 300 samples
Epoch 1/10
700/700 [==============================] - 127s 182ms/step - loss: 6.2359 - accuracy: 0.2154 - val_loss: 4.6726 - val_accuracy: 0.1123
Epoch 2/10
700/700 [==============================] - 125s 178ms/step - loss: 2.5769 - accuracy: 0.4908 - val_loss: 5.5044 - val_accuracy: 0.1029
Epoch 3/10
700/700 [==============================] - 131s 187ms/step - loss: 2.1440 - accuracy: 0.5078 - val_loss: 5.2780 - val_accuracy: 0.2072
Epoch 4/10
700/700 [==============================] - 124s 177ms/step - loss: 1.8707 - accuracy: 0.5753 - val_loss: 9.1681 - val_accuracy: 0.1688
Epoch 5/10
700/700 [==============================] - 124s 177ms/step - loss: 1.9205 - accuracy: 0.5815 - val_loss: 13.6565 - val_accuracy: 0.1131
Epoch 6/10
700/700 [==============================] - 124s 177ms/step - loss: 1.7527 - accuracy: 0.5915 - val_loss: 10.3522 - val_accuracy: 0.1626
Epoch 7/10
700/700 [==============================] - 129s 184ms/step - loss

In [22]:
predictions = model.predict(x)

In [32]:
tp = 0
tn = 0
fp = 0
fn = 0
for i in range(len(y)):
    for j in range(len(y[i])):
        actual = round(y[i][j][0])
        pred = round(predictions[i][j][0])
        if actual == 0 and pred == 0:
            tn += 1
        elif actual == 0 and pred > 0:
            fp += 1
        elif actual > 0 and pred == 0:
            fn += 1
        else:
            tp += 1
total = tp+tn+fp+fn

(3692391, 1044351, 195801, 67457)

In [34]:
print("Accuracy:",(tp+tn)*100/total)
print("True Positive Rate:",tp*100/total)
print("True Negative Rate:",tn*100/total)
print("False Postiive Rate:",fp*100/total)
print("False Negative Rate:",fn*100/total)

Accuracy: 94.73484
True Positive Rate: 73.84782
True Negative Rate: 20.88702
False Postiive Rate: 3.91602
False Negative Rate: 1.34914


In [38]:
p_error = 0
n_error = 0
for i in range(len(y)):
    for j in range(len(y[i])):
        actual = y[i][j][0]
        pred = predictions[i][j][0]
        if round(actual) == 0:
            n_error += abs(actual-pred)
        else:
            p_error += abs(actual-pred)

In [39]:
print("Average Error:",(p_error+n_error)/total)
print("Actual Positive Average Error:",p_error/total)
print("Actual Negative Average Error:",n_error/total)

Average Error: 1.320657059439701
Actual Positive Average Error: 1.2318114114205538
Actual Negative Average Error: 0.08884564801914692
